#### HyerTS 期待什么格式的数据？



**摘要：** 在本NoteBook文档中，为了您可以快速掌握HyperTS的使用并顺利获取实验报告，我们将详细地介绍HyperTS在各类时序任务中所需的规范化数据格式。

**关键词**：DataFrame; Nested; TimeStamp; Covariables.

### 1. 预测任务(Forecasting)

在预测任务中，输入数据应该是一个含有时间列(TimeStamp)和变量列的```pandas DataFrame```格式的二维数据表,形式如下所示：

```python
     time_col          var_col_0        var_col_1        var_col_2     ...    var_col_n
xxxx-xx-xx xx:xx:xx        x                x                x                    x
xxxx-xx-xx xx:xx:xx        x                x                x                    -
xxxx-xx-xx xx:xx:xx        x                x                x                    x
xxxx-xx-xx xx:xx:xx        -                x                x                    x
xxxx-xx-xx xx:xx:xx        x                x                x                    x
xxxx-xx-xx xx:xx:xx        x                -                x                    x
xxxx-xx-xx xx:xx:xx        x                -                x                    x
xxxx-xx-xx xx:xx:xx        x                x                -                    x
xxxx-xx-xx xx:xx:xx        x                x                x                    x
xxxx-xx-xx xx:xx:xx        x                x                -                    x
xxxx-xx-xx xx:xx:xx        -                -                -                    -
xxxx-xx-xx xx:xx:xx        x                x                x                    x
        -                  -                -                -                    -
xxxx-xx-xx xx:xx:xx        x                x                x                    x
```

**其中，xxxx-xx-xx xx:xx:xx表示时间，(x)表示某个时刻某个变量值，(-)表示缺失值。**

**注意事项**

- 在预测任务中的数据中，HyperTS期待也必须含有时间列，列名称不作规范，无论是```ds, ts, timestamp, TimeStamp```还是其他。
- 时间列可以具有```pandas.to_datetime```可以识别的任何格式。
- 时间可以是无序的，HyperTS可以自动转化为有序数列。
- 输入数据的频率满足多种时间粒度，秒(S)、分钟(T)、小时(H)、日(D)、周(W)、月(M)、每年(Y)等等。
- 输入数据容忍存在缺失值，缺失点与缺失时间片段, HyperTS将会在数据预处理过程被填充。
- 输入数据容忍存在重复行，重复的时间片段, HyperTS将会在被在数据预处理过程被裁剪。

当所解决任务中有辅助建模的数据，我们称之为协变量，它仅需跟附在上述数据的```DataFrame```中，形式如下：

```python
     time_col          var_col_0   var_col_1 ... var_col_n     covar_col_0    covar_col_1 ... covar_col_m
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x 
xxxx-xx-xx xx:xx:xx        x          x              -              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              -               x
xxxx-xx-xx xx:xx:xx        -          x              x              x              x               -
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          -              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          -              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              -               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
xxxx-xx-xx xx:xx:xx        -          -              -              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
        -                  -          -              -              -              -               -
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
```
**其中，covar_col_i (i=1,2,..,m)表示协变量。**

**注意事项**

- 协变量可以是数值型(连续变量)也可以是类别型(离散变量)。
- 协变量也容忍存在缺失值和重复值，不需要自己处理。

**举个例子**

In [1]:
import numpy as np
import pandas as pd

size=5

#没有协变量
df_no_covariate = pd.DataFrame({
    'timestamp': pd.date_range(start='2022-02-01', periods=5, freq='H'),
    'val_0': np.random.normal(size=size),
    'val_1': [0.5, 0.2, np.nan, 0.9, 0.0],
    'val_2': np.random.normal(size=size),
})

df_no_covariate

,timestamp,val_0,val_1,val_2
0,2022-02-01 00:00:00,-0.082988,0.5,-2.013005
1,2022-02-01 01:00:00,-0.413941,0.2,0.684124
2,2022-02-01 02:00:00,-0.503602,NaN,0.261157
3,2022-02-01 03:00:00,-0.955423,0.9,-0.125037
4,2022-02-01 04:00:00,0.247220,0.0,0.148305


In [2]:
# 有协变量
df_with_covariate = pd.DataFrame({
    'timestamp': pd.date_range(start='2022-02-01', periods=size, freq='D'),
    'val_0': np.random.normal(size=size),
    'val_1': [12, 52, 34, np.nan, 100],
    'val_2': [0.5, 0.2, np.nan, 0.9, 0.0],
    'covar_0': [0.2, 0.4, 0.2, 0.7, 0.1],
    'covar_1': ['a', 'a', 'b', 'b', 'b'],
    'covar_2': [1, 2, 2, None, 3], 
})

df_with_covariate

,timestamp,val_0,val_1,val_2,covar_0,covar_1,covar_2
0,2022-02-01,-1.145418,12.0,0.5,0.2,a,1.0
1,2022-02-02,-1.466120,52.0,0.2,0.4,a,2.0
2,2022-02-03,0.116391,34.0,NaN,0.2,b,2.0
3,2022-02-04,-0.608921,NaN,0.9,0.7,b,NaN
4,2022-02-05,-1.250787,100.0,0.0,0.1,b,3.0


**加载HyperTS的数据集**

In [3]:
from hyperts.datasets import (load_random_univariate_forecast_dataset, 
                              load_random_multivariate_forecast_dataset,
                              load_network_traffic)

In [4]:
df0 = load_random_univariate_forecast_dataset(return_X_y=False)
df0.head()

,ds,id,value
0,2013-01-01,0.0,0.922501
1,2013-01-02,0.0,0.496957
2,2013-01-03,1.0,0.252253
3,2013-01-04,0.0,0.103628
4,2013-01-05,0.0,0.853269


在这个数据集中，```ds```是时间列名称，时间频率是天(D), 目标变量(预测变量)是```value```列，```id```列是协变量列，是一个单变量时间序列任务。

In [5]:
df1 = load_random_multivariate_forecast_dataset(return_X_y=False)
df1.head()

,ds,Var_1,Var_2
0,2022-02-10 14:32:05.430194,0.461891,0.866003
1,2022-02-11 14:32:05.430194,1.761956,2.608567
2,2022-02-12 14:32:05.430194,2.591447,2.807734
3,2022-02-13 14:32:05.430194,3.622879,4.596779
4,2022-02-14 14:32:05.430194,4.098946,4.340272


在这个数据集中，```ds```是时间列名称，时间频率是天(D), 目标变量(预测变量)是```Val_1, Val_2```两列，没有协变量，是一个多变量时间序列任务。

In [6]:
df2 = load_network_traffic(return_X_y=False)
df2.head()

,TimeStamp,Var_1,Var_2,Var_3,Var_4,Var_5,Var_6,HourSin,WeekCos,CBWD
0,2021-03-01 00:00:00,0.7534,3.375,10.195,1.4490,19174.977,286443.880,0.000000,1.0,NW
1,2021-03-01 01:00:00,0.3376,2.414,3.920,0.4065,7529.263,178930.450,0.258819,1.0,NW
2,2021-03-01 02:00:00,0.2032,1.654,3.318,0.2142,3310.539,42296.164,0.500000,1.0,NW
3,2021-03-01 03:00:00,0.2420,1.393,3.148,0.2312,4535.464,26220.232,0.707107,1.0,NW
4,2021-03-01 04:00:00,0.1940,1.429,3.215,0.2157,2732.911,27990.348,0.866025,1.0,NW


在这个数据集中，```TimeStamp```是时间列名称，时间频率是小时(H), 目标变量(预测变量)是```Val_1, Val_2，Val_3, Val_4，Val_5, Val_6```六列，协变量是HourSin, WeekCos, CBWD, 其中HourSin, WeekCos是数值型变量，CBWD是类别型变量，是一个多变量时间序列任务。

<bar>
<bar>

### 2. 分类任务(Classification/Regression)

具体表现在输入数据的形式为含有目标列(target)及特征列的嵌套(nested) ```pandas DataFrame```格式的二维数据表，形式如下所示：

```python
 var_col_0       var_col_1   ...    var_col_n     target
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      0
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      0
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      1
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      1
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      1
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      2
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      2
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      2
x,x,x,...,x     x,x,x,...,x        x,x,x,...,x      2
```

**其中，x,x,x,...,x表示某样本在len(x,x,x,...,x)长度的时间片段某变量随时间的波动情况。(x)表示某个时刻某个变量值。**

**注意事项**

- 分类或者回归任务是针对一个样本判断其行为，故与预测任务的数据形式不同，预测数据每一行表示一个时间点各个变量的值，而分类或回归数据每一行表示一个样本，而每一个cell, 即```x,x,x,...,x```表示某样本在len(```x,x,x,...,x```)长度的时间片段某变量随时间波动的情况。每个样本根据各个变量的序列行为判别```target```。
- 直觉上，```pandas DadaFrame```是一二维数据表，每一个cell储存一个数值，现在我们储存一个序列，从而将三维数据嵌套在二维数据表中，这也是我们称之为 ```nested DataFrame```的原因。
- 分类或回归任务的目标是判别每一个样本的类别或者行为，故数据的走势是关键特质，所以为了简单起见，我们在存储时省略去TimeStamp的信息。

**举个例子**

In [7]:
import numpy as np
import pandas as pd

size=10

df = pd.DataFrame({
    'var_0': [pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size)),
              pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size)),
              pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size))],
    'var_1': [pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size)),
              pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size)),
              pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size))],
    'var_2': [pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size)),
              pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size)),
              pd.Series(np.random.normal(size=size)), pd.Series(np.random.normal(size=size))],
    'y': [0, 0, 1, 1, 2, 2], 
})

df

,var_0,var_1,var_2,y
0,0 1.517646 1 -0.130092 2 0.041960 3 ...,0 1.339915 1 0.905619 2 0.661068 3 ...,0 0.458820 1 -0.205814 2 -0.779682 3 ...,0
1,0 -2.202468 1 0.586854 2 0.741383 3 ...,0 1.792085 1 0.169211 2 -1.507392 3 ...,0 -0.191859 1 1.491976 2 1.643508 3 ...,0
2,0 -0.257921 1 0.277949 2 0.113696 3 ...,0 0.862955 1 -2.383729 2 0.685731 3 ...,0 -0.481607 1 -3.252144 2 0.705657 3 ...,1
3,0 0.474522 1 0.812421 2 0.945904 3 ...,0 -0.689202 1 -1.111300 2 -0.905664 3 ...,0 -0.922138 1 0.288886 2 -1.289592 3 ...,1
4,0 -0.838331 1 -0.445160 2 -0.417691 3 ...,0 1.117217 1 -0.482835 2 1.816897 3 ...,0 0.255738 1 -0.702608 2 3.110447 3 ...,2
5,0 0.342809 1 -0.018744 2 -1.681167 3 ...,0 0.527534 1 1.360901 2 0.865364 3 ...,0 0.978863 1 0.581206 2 -1.267197 3 ...,2


**加载HyperTS的数据集**

In [8]:
from hyperts.datasets import load_arrow_head, load_basic_motions

In [9]:
df0 = load_arrow_head(return_X_y=False)
df0.head()    

,Var_1,target
0,0 -1.9630 1 -1.9578 2 -1.9561 3 ...,0
1,0 -1.7746 1 -1.7740 2 -1.7766 3 ...,1
2,0 -1.8660 1 -1.8420 2 -1.8350 3 ...,2
3,0 -2.0738 1 -2.0733 2 -2.0446 3 ...,0
4,0 -1.7463 1 -1.7413 2 -1.7227 3 ...,1


In [10]:
df0.target.unique()

array(['0', '1', '2'], dtype=object)

在这个数据集中，```target```是目标列, 包含三个类别['0', '1', '2']， ```Var_1```是特征变量，有且仅有一个，故是一个单变量多分类任务。

In [11]:
df1 = load_basic_motions(return_X_y=False)
df1.head()

,Var_1,Var_2,Var_3,Var_4,Var_5,Var_6,target
0,0 0.079106 1 0.079106 2 -0.903497 3...,0 0.394032 1 0.394032 2 -3.666397 3...,0 0.551444 1 0.551444 2 -0.282844 3...,0 0.351565 1 0.351565 2 -0.095881 3...,0 0.023970 1 0.023970 2 -0.319605 3...,0 0.633883 1 0.633883 2 0.972131 3...,standing
1,0 0.377751 1 0.377751 2 2.952965 3...,0 -0.610850 1 -0.610850 2 0.970717 3...,0 -0.147376 1 -0.147376 2 -5.962515 3...,0 -0.103872 1 -0.103872 2 -7.593275 3...,0 -0.109198 1 -0.109198 2 -0.697804 3...,0 -0.037287 1 -0.037287 2 -2.865789 3...,standing
2,0 -0.813905 1 -0.813905 2 -0.424628 3...,0 0.825666 1 0.825666 2 -1.305033 3...,0 0.032712 1 0.032712 2 0.826170 3...,0 0.021307 1 0.021307 2 -0.372872 3...,0 0.122515 1 0.122515 2 -0.045277 3...,0 0.775041 1 0.775041 2 0.383526 3...,standing
3,0 0.289855 1 0.289855 2 -0.669185 3...,0 0.284130 1 0.284130 2 -0.210466 3...,0 0.213680 1 0.213680 2 0.252267 3...,0 -0.314278 1 -0.314278 2 0.018644 3...,0 0.074574 1 0.074574 2 0.007990 3...,0 -0.079901 1 -0.079901 2 0.237040 3...,standing
4,0 -0.123238 1 -0.123238 2 -0.249547 3...,0 0.379341 1 0.379341 2 0.541501 3...,0 -0.286006 1 -0.286006 2 0.208420 3...,0 -0.098545 1 -0.098545 2 -0.023970 3...,0 0.058594 1 0.058594 2 0.175783 3...,0 -0.074574 1 -0.074574 2 0.114525 3...,standing


In [12]:
df1.target.unique()

array(['standing', 'running', 'walking', 'badminton'], dtype=object)

在这个数据集中, ```target```是目标列，包含四个类别['standing', 'running', 'walking', 'badminton'], ```Var_1, Var_2, Var_3, Var_4, Var_5, Var_6```是特征变量，共六个，故是一个多变量多分类任务。

**提示**

当拿到的原始数据是```numpy.array```形式时，我们如何将其转化为嵌套的```pandas.DataFrame```数据呢？例如如下数据：

In [13]:
import numpy as np

nb_samples = 100
series_length = 72
nb_variables = 6
nb_classes = 4

X = np.random.normal(size=nb_samples*series_length*nb_variables).reshape(nb_samples, series_length, nb_variables)
y = np.random.randint(low=0, high=nb_classes, size=nb_samples)

In [14]:
X.shape, y.shape, np.unique(y)

((100, 72, 6), (100,), array([0, 1, 2, 3]))

由数据可知，该数据包含了```100```个样本，每个样本有```6```个变量，而每个变量是长度为```72```的时间序列。y共有```4```个类别。

面对这样的情况，HyperTS提供了相关变换的工具函数```from_3d_array_to_nested_df```：

In [15]:
import pandas as pd
from hyperts.toolbox import from_3d_array_to_nested_df

df_X = from_3d_array_to_nested_df(data=X)
df_y = pd.DataFrame({'y': y})
df = pd.concat([df_X, df_y], axis=1)

In [16]:
df.head()

,Var_0,Var_1,Var_2,Var_3,Var_4,Var_5,y
0,0 -0.860411 1 1.862670 2 -0.921505 3...,0 1.398394 1 0.660091 2 0.128765 3...,0 0.525402 1 -0.308939 2 -0.404762 3...,0 0.300226 1 0.111569 2 -0.659606 3...,0 0.245592 1 1.196472 2 0.037356 3...,0 -0.192188 1 -0.894914 2 0.590388 3...,3
1,0 0.715266 1 0.315416 2 0.154454 3...,0 0.176125 1 -1.172978 2 0.456093 3...,0 1.041922 1 -0.385470 2 0.916923 3...,0 0.618068 1 -0.830899 2 -2.476492 3...,0 1.242938 1 -0.375228 2 -1.948744 3...,0 0.309130 1 0.872872 2 -0.512654 3...,0
2,0 -0.190163 1 -0.636367 2 0.557982 3...,0 -0.304956 1 0.346306 2 -0.312518 3...,0 0.031167 1 0.154234 2 -0.368498 3...,0 0.038083 1 -0.346858 2 -0.121023 3...,0 0.977429 1 0.084071 2 0.226736 3...,0 -0.225884 1 -0.653812 2 0.392387 3...,2
3,0 -0.805321 1 -0.985039 2 0.747402 3...,0 -0.750069 1 -0.495475 2 -0.335203 3...,0 1.020891 1 -1.145493 2 0.289771 3...,0 1.354475 1 -1.320292 2 -1.861200 3...,0 -0.189189 1 1.339734 2 -0.354420 3...,0 0.762758 1 0.559134 2 -1.123743 3...,2
4,0 -0.358629 1 -1.338477 2 0.547015 3...,0 -1.078313 1 -0.145033 2 0.044039 3...,0 0.122722 1 -0.038923 2 -1.711621 3...,0 0.212634 1 0.379226 2 -1.555363 3...,0 -0.348793 1 -1.117400 2 -1.463423 3...,0 2.460091 1 0.306163 2 -0.568233 3...,3


### 3. 异常检测任务(Anomaly Detection)

与预测任务相似，输入数据应该是一个含有时间列(TimeStamp)和变量列的```pandas DataFrame```格式的二维数据表, 其应该包含时间戳列(``time_col``)，一个或多个变量列(``var_col_0``, ``var_col_1``, ``var_col_2``,... ``var_col_n``)，如果有协变量，也可包含一个或多个协变量(``covar_col_0``, ``covar_col_1``, ``covar_col_2``,... ``covar_col_m``)，形式如下所示：

```python
     time_col          var_col_0   var_col_1 ... var_col_n     covar_col_0    covar_col_1 ... covar_col_m
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x 
xxxx-xx-xx xx:xx:xx        x          x              -              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              -               x
xxxx-xx-xx xx:xx:xx        -          x              x              x              x               -
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          -              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          -              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              -               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
xxxx-xx-xx xx:xx:xx        -          -              -              x              x               x
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
        -                  -          -              -              -              -               -
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x
```

此外，以上数据也可以包含真实标签，这将有助于模型选择和超参数搜索过程。形式如下所示:

```python
     time_col          var_col_0   var_col_1 ... var_col_n     covar_col_0    covar_col_1 ... covar_col_m       anomaly       
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x               1 
xxxx-xx-xx xx:xx:xx        x          x              -              x              x               x               0
xxxx-xx-xx xx:xx:xx        x          x              x              x              -               x               0
xxxx-xx-xx xx:xx:xx        -          x              x              x              x               -               0
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x               1
xxxx-xx-xx xx:xx:xx        x          -              x              x              x               x               0
xxxx-xx-xx xx:xx:xx        x          -              x              x              x               x               0
xxxx-xx-xx xx:xx:xx        x          x              x              x              -               x               1
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x               0
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x               0
xxxx-xx-xx xx:xx:xx        -          -              -              x              x               x               1
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x               0
        -                  -          -              -              -              -               -
xxxx-xx-xx xx:xx:xx        x          x              x              x              x               x               0
```


where ``anomaly`` is anomaly label column.

**注意事项**

   当数据包含真实标签时，优化评估采用真实标签。否则，生成的伪标签将被采用。